In [1]:
import pandas as pd
import numpy as np

from scipy.stats.stats import pearsonr
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
# Load features from the CNN model
train_feat_max = pd.read_csv('/project/DSone/as3ek/data/csv_files/seem_train_2048_features_max_normal_all__ee_vsi.csv')
valid_feat_max = pd.read_csv('/project/DSone/as3ek/data/csv_files/seem_valid_2048_features_max_normal_all__ee_vsi.csv')

In [4]:
# Load cleaned EE RNA Seq Data
ee_rna_seq = pd.read_csv('/project/DSone/as3ek/data/csv_files/ee_trascript.csv')
ee_rna_seq = ee_rna_seq.drop(['Unnamed: 0'], axis=1)

In [5]:
# Load RNA Seq Data
rna_seq = pd.read_csv('/project/DSone/as3ek/data/csv_files/transcriptome_merged.csv')

# Drop unnecessary columns
rna_seq = rna_seq.drop(['Gene', 'PID', 'RNASes_Bx_loc', 'SEEM ID', 'Bx Date', 'Accession #'], axis=1)

# Filter all data to get only normal RNA Seq data
normal_rna_seq = rna_seq[rna_seq['Diagnosis'] == 'CONTROL'].reset_index(drop=True)
normal_rna_seq = normal_rna_seq.drop('Diagnosis', axis=1)

/apps/software/standard/core/anaconda/5.2.0-py3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13468,13469,13470) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
ee_rna_seq.head()

,A1BG,A1CF,A2M,A4GALT,A4GNT,AAAS,AACS,AADAC,AAED1,AAGAB,...,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3,FileNames
0,2.938328,19.990509,83.735050,2.656216,3.404470,17.141710,11.889671,57.604313,2.052005,22.352093,...,7.167301,6.760921,23.934557,1.351300,5.770310,2.624760,32.301320,16.912931,10.427124,SEEM_1
1,4.698266,15.687117,92.621060,1.957990,0.306087,12.404878,9.844436,84.911210,1.641775,21.663334,...,7.521921,6.561717,18.160603,1.151140,6.536171,3.283770,34.461420,14.909936,13.230409,SEEM_3
2,3.455630,13.785192,107.816505,3.281972,0.010000,13.406025,8.643127,49.870205,2.645186,18.025269,...,4.720486,4.125326,13.924426,0.878572,4.932373,2.385005,29.004587,15.764729,6.827882,SEEM_4
3,2.466141,36.609490,135.728450,3.121654,0.293090,17.032896,12.018597,91.219810,3.087415,23.395113,...,8.026385,7.405385,14.101945,2.055490,7.218950,4.769273,30.454320,25.666950,14.545485,SEEM_5
4,3.249304,30.674992,115.543710,2.711289,1.881430,19.385916,11.410261,96.130104,1.817835,18.924387,...,6.723240,6.889716,18.950125,1.843850,7.449580,3.398000,22.919546,26.630081,12.240758,SEEM_6


In [7]:
normal_rna_seq.head()

,A1BG,A1CF,A2M,A4GALT,A4GNT,AAAS,AACS,AADAC,AAED1,AAGAB,...,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3,FileNames
0,1.527489,39.161606,141.393680,2.250741,3.340980,13.393648,11.521561,190.26294,3.802879,27.817423,...,7.966211,5.824094,15.380531,1.90331,6.084198,5.986877,19.513960,30.423355,13.065509,GI-17-2781A
1,1.836630,20.815561,70.397380,1.366245,8.056910,13.729294,11.792672,188.86170,2.597826,19.777740,...,6.067741,3.992341,12.340939,1.04346,4.722662,2.653590,24.902603,14.547653,10.382214,GI-17-5415A
2,2.070108,30.049267,78.129480,2.200778,8.668561,14.197790,13.160649,299.86722,2.576309,22.687897,...,6.487394,4.760917,12.637662,1.09973,5.131794,4.623030,23.566916,17.086746,8.982910,GI-17-5671A
3,2.323137,27.274525,93.843880,2.508640,1.156290,15.076123,11.050514,176.36838,1.392020,19.399889,...,4.988570,2.749776,13.213606,1.35546,5.087205,2.442820,26.665813,20.346804,7.571774,GI-17-6710A
4,1.550304,28.912386,78.544624,2.113191,0.260323,14.713420,10.754949,272.44060,2.326984,22.267065,...,7.437720,5.820460,18.061235,1.39190,5.067470,3.432585,24.128992,15.404562,10.151729,GI-17-6728A


In [8]:
# Concat and save clean RNA seq data
rna_seq_clean = pd.concat([ee_rna_seq, normal_rna_seq], ignore_index=True)
rna_seq_clean.to_csv('/project/DSone/as3ek/data/csv_files/rna_seq_clean_normal_ee.csv', index=False)

In [9]:
# Filter features to get EE data and merge from train and valid
ee_train_feat_max = train_feat_max[train_feat_max['fname'].str.contains('SEEM')].reset_index(drop=True)
ee_valid_feat_max = valid_feat_max[valid_feat_max['fname'].str.contains('SEEM')].reset_index(drop=True)
ee_feat_max = pd.concat([ee_train_feat_max, ee_valid_feat_max])

# Clean the name field
ee_feat_max['fname'] = ee_feat_max['fname'].str.split('__').str[0].str.split('_').str[0] + '_' + ee_feat_max['fname'].str.split('__').str[0].str.split('_').str[1]

In [10]:
# Filter features to get Normal data and merge from train and valid
normal_train_feat_max = train_feat_max[train_feat_max['fname'].str.contains('G')].reset_index(drop=True)
normal_valid_feat_max = valid_feat_max[valid_feat_max['fname'].str.contains('G')].reset_index(drop=True)
normal_feat_max = pd.concat([normal_train_feat_max, normal_valid_feat_max], ignore_index=True)

# Clean the name field
normal_feat_max['fname'] = normal_feat_max['fname'].str.split('_').str[0].str.replace('GI', 'GI-').str.split(' ').str[0] + 'A'

In [11]:
# Group features for all patches by biopsy name
ee_feat_max_grpd = ee_feat_max.groupby('fname', as_index=False).mean()
normal_feat_max_grpd = normal_feat_max.groupby('fname', as_index=False).mean()

In [54]:
ee_feat_max_grpd.head()

,fname,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,SEEM_1,1.400954,2.726676,1.868812,1.917903,2.655619,2.859661,2.283474,2.000574,1.911550,...,1.892482,1.881015,1.763218,1.135075,3.023325,0.631242,2.081944,1.175545,1.302622,1.588411
1,SEEM_10,1.347156,3.181994,2.297989,1.938909,2.182216,2.793844,2.065651,2.069440,1.613876,...,2.755671,2.465041,2.863062,1.014355,3.783387,0.726808,2.891178,1.032845,1.699797,2.078044
2,SEEM_11,1.093749,3.349344,3.042262,1.958739,1.869790,2.074053,1.838703,2.658418,1.740939,...,3.729062,3.444447,2.597348,0.715201,4.651663,0.684788,3.045861,0.770555,1.404238,3.087810
3,SEEM_12,1.034657,3.299655,2.598511,2.353304,1.631794,2.352409,1.752420,2.765432,1.396725,...,3.285949,2.576968,3.049833,0.748103,3.937245,0.978531,2.349350,0.777661,1.627161,2.904910
4,SEEM_13,1.146579,2.898013,2.440671,2.175549,2.432416,2.872315,1.321863,2.363730,2.061244,...,3.272165,2.882773,3.017845,0.867513,4.961912,0.632133,3.110673,0.687398,2.304506,2.189863


In [52]:
normal_feat_max_grpd.head()

,fname,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,GI-17-10289A,1.705304,1.308385,1.214186,1.144616,2.019404,1.173569,1.481161,1.411003,1.604710,...,1.353803,1.501212,1.755425,2.285431,1.236964,1.350781,1.407560,1.979489,3.193440,1.053154
1,GI-17-10498A,1.791951,1.338543,1.063339,0.861534,2.173868,1.278980,0.839176,1.369282,2.094078,...,1.764162,1.548570,1.801140,2.439009,1.137720,1.467445,1.001710,2.256842,3.805646,0.982195
2,GI-17-10706A,1.624928,1.510126,1.291649,1.120004,2.066083,1.305926,1.127223,1.419618,1.683323,...,1.777002,1.732200,1.997007,2.087791,1.456396,1.212907,1.362405,1.793345,3.366465,1.193909
3,GI-17-2781A,1.737263,1.149561,1.115752,0.711680,1.691020,0.998992,1.301657,1.142493,1.653945,...,1.295423,1.388237,1.726299,2.544281,1.158321,1.524205,1.003201,2.495913,3.502311,0.951937
4,GI-17-5073A,1.639812,1.273617,1.188617,0.867676,2.264301,1.224853,0.857378,1.247174,2.027124,...,1.428486,1.408774,1.793765,2.605442,1.279758,1.315401,1.038976,2.253473,3.944400,0.843002


In [14]:
feat_max_grpd_clean = pd.concat([ee_feat_max_grpd, normal_feat_max_grpd], ignore_index=True)
feat_max_grpd_clean.to_csv('/scratch/as3ek/histo_visual_recog/scripts/data/max_features_grouped_clean_SEEM_GI.csv', index=False)

In [56]:
ee_seq_feat.shape

(46, 15514)

In [57]:
normal_seq_feat.shape

(14, 15514)

In [15]:
# Merge seq with features
ee_seq_feat = pd.merge(ee_rna_seq, ee_feat_max_grpd, left_on='FileNames', right_on='fname')
normal_seq_feat = pd.merge(normal_rna_seq, normal_feat_max_grpd, left_on='FileNames', right_on='fname')

In [16]:
# Concate EE and normal frames
seq_feat = pd.concat([ee_seq_feat, normal_seq_feat], ignore_index=True)
seq_feat.to_csv('/scratch/as3ek/histo_visual_recog/results/rnaseq_feat.csv', index=False)

In [18]:
# Getting list of all columns for correlation b/w
x = list(ee_rna_seq)
x.remove('FileNames')

y = list(ee_feat_max_grpd)
y.remove('fname')

In [19]:
# Calculate pairwise correlation and filter it for reqd columns and round the values
corr = seq_feat.corr()
corr_fltrd = corr[x][corr.index.isin(y)]
corr_fltrd = corr_fltrd.round(4)

In [20]:
corr_fltrd.to_csv('/scratch/as3ek/histo_visual_recog/results/gene_feat_corr.csv', index=False)

In [100]:
gene_map = pd.read_csv('/scratch/as3ek/histo_visual_recog/scripts/data/custom.txt', sep='\t')

In [25]:
def corr_cutoff(corr_fltrd, cutoff):
    # Make copy
    df = corr_fltrd.copy().T
    
    # Replace all values below cutoff with 0
    for col in y:
        df.loc[np.abs(df[col]) < cutoff, col] = 0
    
    # Remove all columns with all zeros (Remove unnecessary features)
    df = df.loc[:, (df != 0).any(axis=0)]
    
    # Transpose and remove all columns with all zeros (Remove unnecessary genes)
    df = df.T
    df = df.loc[:, (df != 0).any(axis=0)]
    
    return df

In [133]:
def compute_corr_metrics(corr_fltrd_cutoff, seq_feat, gene_map):
    corr_metrics = pd.DataFrame(columns=['Gene Symbol', 'Gene Name', 'Feature', 'PearsonR', 'p-value'])
    c = corr_fltrd_cutoff.copy()
    for gene in c.columns:
        df = pd.DataFrame(c[gene]).T
        df = df.loc[:, (df != 0).any(axis=0)]
        
        names = list(gene_map[gene_map['Approved symbol'] == gene]['Approved name'])
        if len(names) > 0:
            name = names[0]
        else:
            name = gene
        
        for feat in df.columns:
            r, p = pearsonr(seq_feat[feat], seq_feat[gene])
            tmp = pd.DataFrame([[gene, name, feat, r, p]], columns=['Gene Symbol', 'Gene Name', 'Feature', 'PearsonR', 'p-value'])
            corr_metrics = pd.concat([corr_metrics, tmp], ignore_index=True)
    return corr_metrics

In [127]:
def compute_regulation(normal_rna_seq, ee_rna_seq, genes, cutoff_fold, gene_map):
    regulation_data = pd.DataFrame(columns=['Regulation', 'Gene Symbol', 'Gene Name', 'fold'])
    for col in genes:
        a = np.mean(ee_rna_seq[col]) / np.mean(normal_rna_seq[col]) 
        b = 1 / a
        names = list(gene_map[gene_map['Approved symbol'] == col]['Approved name'])
        if len(names) > 0:
            name = names[0]
        else:
            name = col
        if np.abs(a) > cutoff_fold:
            tmp = pd.DataFrame([['up', col, name, a]], columns=['Regulation', 'Gene Symbol', 'Gene Name', 'fold'])
            regulation_data = pd.concat([regulation_data, tmp], ignore_index=True)
        if np.abs(b) > cutoff_fold:
            tmp = pd.DataFrame([['down', col, name, b]], columns=['Regulation', 'Gene Symbol', 'Gene Name', 'fold'])
            regulation_data = pd.concat([regulation_data, tmp], ignore_index=True)
    
    return regulation_data

In [128]:
regulation_data = compute_regulation(normal_rna_seq, ee_rna_seq, x, 1, gene_map)

In [129]:
regulation_data.to_csv('/scratch/as3ek/histo_visual_recog/scripts/data/gene_regulation_data.csv', index=False)

In [139]:
corr_fltrd_cutoff = corr_cutoff(corr_fltrd, 0.6)

In [ ]:
plt.figure(figsize=(30, 200))
sns.heatmap(corr_fltrd_cutoff)

In [141]:
corr_metrics = compute_corr_metrics(corr_fltrd_cutoff, seq_feat, gene_map)
corr_metrics.to_csv('/scratch/as3ek/histo_visual_recog/scripts/data/corr_metric_cutoff_6.csv', index=False)

In [152]:
from sklearn.cluster import KMeans

In [205]:
kmeans_all_data = KMeans(n_clusters=5, random_state=0).fit(corr_fltrd)
kmeans_cutoff_data = KMeans(n_clusters=5, random_state=0).fit(df_cl)

In [206]:
corr_fltrd['cluster'] = kmeans_all_data.labels_
df_cl['cluster'] = kmeans_cutoff_data.labels_

In [219]:
feat_cluster = pd.DataFrame()

for cluster in np.unique(df_cl['cluster']):
    cl = df_cl[df_cl['cluster'] == cluster].drop('cluster', axis=1).copy().T
    
    # Remove all 0 values from columns
    cl = cl.loc[:, (cl != 0).any(axis=0)]

    # Remove all 0 values from rows
    cl = cl[(cl.T != 0).any()]
    
    tmp = pd.DataFrame([cluster, list(cl.index), len(cl.index), list(cl.columns), len(cl.columns)]).T
    feat_cluster = pd.concat([feat_cluster, tmp])

In [221]:
feat_cluster.columns = ['cluster', 'genes', 'num_genes', 'features', 'num_features']

In [222]:
feat_cluster.to_csv('/scratch/as3ek/histo_visual_recog/results/feature_clusters_cutoff_8.csv', index=False)

In [226]:
np.min(corr_fltrd['DUOXA2'])

-0.44

In [235]:
np.mean(normal_rna_seq[np.unique(sgnfct_seq_feat_corr['gene'])[0]])

4035.613696

In [234]:
np.mean(ee_rna_seq[np.unique(sgnfct_seq_feat_corr['gene'])[0]])

2174.0717784313724

In [233]:
np.unique(sgnfct_seq_feat_corr['gene'])[0]

array(['ALDOB', 'AMACR', 'APOH', 'CA2', 'CAPN13', 'CDK20', 'CHAD',
       'CRIP3', 'CYP4F3', 'EPHX1', 'HGD', 'IGSF9', 'ISX', 'IYD', 'KCNK10',
       'KIF12', 'LTK', 'MAPKAPK3', 'ME1', 'MMP24', 'MMP28', 'MOCS1',
       'NELL2', 'OAT', 'PPP1R36', 'RAPGEFL1', 'RGS11', 'SEMA6C', 'SHMT1',
       'SLC5A4', 'TMEM229A', 'TMEM25', 'TMEM52', 'UNC5D', 'UNC93A',
       'UPB1', 'USP2'], dtype=object)

In [236]:
normal_rna_seq[np.unique(sgnfct_seq_feat_corr['gene'])[0]]

0     3672.1885
1     4264.6830
2     3090.2253
3     4540.7686
4     3200.6025
5     3714.6306
6     5261.3057
7     4697.8080
8     6668.3076
9     2550.5593
10    4364.0550
11    4898.0825
12    6355.1100
13    5055.7250
14    3362.0535
15    2191.2117
16    2555.9316
17    4323.9400
18    4143.9870
19    5283.7720
20    3237.6394
21    3434.0540
22    3716.8413
23    3360.2893
24    2946.5710
Name: ALDOB, dtype: float64